## Download/Install packages

In [0]:
!mkdir albert
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-config.json -O /content/albert/config.json
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-modelcard.json -O /content/albert/modelcard.json
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-pytorch_model.bin -O /content/albert/pytorch_model.bin
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-spiece.model -O /content/albert/spiece.model
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-tf_model.h5 -O /content/albert/tf_model.h5
!wget https://s3.amazonaws.com/models.huggingface.co/bert/albert-base-v2-with-prefix-tf_model.h5 -O /content/albert/prefix-tf_model.h5
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gsarti/biobert-nli/vocab.txt -O /content/albert/vocab.txt

In [0]:
!pip install -U sentence-transformers
!pip install git+https://github.com/huggingface/transformers
!git clone https://github.com/huggingface/transformers.git 

     |████████████████████████████████| 61kB 2.3MB/s 
     |████████████████████████████████| 645kB 9.9MB/s 
     |████████████████████████████████| 1.1MB 65.1MB/s 
     |████████████████████████████████| 3.8MB 59.4MB/s 
     |████████████████████████████████| 890kB 59.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=ba3da9f72c610fb2ab996e67259858adc12aeb4d1079791211fa4a78747030d9
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ec39ba7a237d2d5756ff48c9e0982333386827bd0d1c461b5c5fe9c70339dd79
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-umtt1mvu
  Running command git clone 

## FineTune

In [0]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("/content/drive/My Drive/COVID Press/covid_press.csv")
df_sentences_list = list(df_sentences["text"])
df_sentences_list = [str(d) for d in df_sentences_list]
print(len(df_sentences_list))
file_content = "\n".join(df_sentences_list)
with open("input_text.txt","w") as f:
  f.write(file_content)

66109


In [0]:
#https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/
#https://huggingface.co/transformers/v2.1.1/examples.html#roberta-bert-and-masked-language-modeling
!python "/content/transformers/examples/language-modeling/run_language_modeling.py" \
    --output_dir="/content/drive/My Drive/COVID Press/BertCovid" \
    --model_type=bert \
    --model_name_or_path=google/bert_uncased_L-8_H-256_A-4 \
    --do_train \
    --block_size=512 \
    --train_data_file="/content/input_text.txt" \
    --mlm

2020-05-13 13:03:50.411120: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/13/2020 13:03:52 - INFO - transformers.training_args -   PyTorch: setting up devices
05/13/2020 13:03:52 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
05/13/2020 13:03:52 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/drive/My Drive/COVID Press/BertCovid', overwrite_output_dir=False, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level

In [0]:
df_sentences_list[:5]


['footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1]  upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth',
 'the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3]  the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4]  which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evo

In [0]:
# if you want more RAMs from google colab
# fill your RAM using this simple loop
a = []
while(1):
    a.append('1')

## Evaluate

In [0]:
from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses, models
from torch.utils.data import DataLoader
from sentence_transformers.readers import TripletReader
from sentence_transformers.evaluation import TripletEvaluator
import csv
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, losses
import scipy.spatial
import pickle as pkl

In [0]:
word_embedding_model = models.ALBERT("/content/albert")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model2 = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
train_batch_size = 16

triplet_reader = TripletReader('/content/drive/My Drive/BertSentenceSimilarity', s1_col_idx=1, s2_col_idx=2, s3_col_idx=3, delimiter=',', quoting=csv.QUOTE_MINIMAL, has_header=True)

test_data = SentencesDataset(examples=triplet_reader.get_examples("result.csv",max_examples=500), model=model2)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = TripletEvaluator(test_dataloader)

model2.evaluate(evaluator)

Evaluating: 100%|██████████| 32/32 [05:56<00:00, 11.15s/it]


0.65

In [0]:
del model2